In [ ]:
import utils.data_getter
from utils.data_getter import TimeSeries
from utils.technical_indicators import calculate_rsi, calculate_adx

In [ ]:
aapl = (
    TimeSeries("AAPL", date="2024-01-01")
    .construct_returns()
    .construct_technical_indicators([calculate_rsi, calculate_adx], [5, 5])
    .lag_column("Returns", skip_lags=1, n_lags=2)
    .dropna()
    .drop_columns(["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
    .train_test_split(test_size=0.2, target="Returns")
)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import xgboost
model = xgboost.XGBRegressor(max_depth = 2, learning_rate = 0.1, n_estimators = 100)
model.fit(aapl.modelling_data.x_train, aapl.modelling_data.y_train)
aapl.predict(model, "Vanilla XGBoost")
MAPE = mean_absolute_percentage_error(aapl.modelling_data.y_test, aapl.predictions["Vanilla XGBoost"])

In [ ]:
print(f"MAPE: {MAPE*100:.2f}%")

## Hyperparameter Optimisation

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }
def objective(space):
    clf=xgboost.XGBRegressor(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( aapl.modelling_data.x_train, aapl.modelling_data.y_train), ( aapl.modelling_data.x_test, aapl.modelling_data.y_test)]
    
    clf.fit(aapl.modelling_data.x_train, aapl.modelling_data.y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(aapl.modelling_data.x_test)
    R2 = r2_score(aapl.modelling_data.y_test, pred)
    return {'loss': -R2, 'status': STATUS_OK }

In [37]:
trials = Trials()
import warnings
warnings.filterwarnings("ignore")
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials,
                        verbose=False)

In [38]:
# Fixing types
best_hyperparams['max_depth'] = int(best_hyperparams['max_depth'])

In [39]:
hyperparam_optimised_model = xgboost.XGBRegressor(**best_hyperparams)
hyperparam_optimised_model.fit(aapl.modelling_data.x_train, aapl.modelling_data.y_train)
aapl.predict(hyperparam_optimised_model, "HPOptimised XGBoost")
MAPE = mean_absolute_percentage_error(aapl.modelling_data.y_test, aapl.predictions["HPOptimised XGBoost"])
R2 = r2_score(aapl.modelling_data.y_test, aapl.predictions["HPOptimised XGBoost"])
print(f"MAPE: {MAPE*100:.2f}%")


MAPE: 96.58%
